<a href="https://colab.research.google.com/github/kannaparthiv/Task-1/blob/main/E_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 pandas numpy scikit-learn matplotlib seaborn plotly flask-ngrok pyngrok

In [2]:
# E-commerce Price Comparison Platform - Google Colab Version
# Comprehensive system optimized for Colab environment

# Install required packages
!pip install requests beautifulsoup4 pandas numpy scikit-learn matplotlib seaborn plotly flask-ngrok pyngrok

import requests
import sqlite3
import json
import hashlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, urlparse
import time
import threading
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, IntSlider, Dropdown, Text, Button, VBox, HBox, Output

print("🚀 E-commerce Price Comparison Platform for Google Colab")
print("=" * 60)

# Data Models (Simplified for Colab)
@dataclass
class Product:
    id: str
    name: str
    category: str
    brand: str
    description: str
    image_url: str
    created_at: datetime = None

    def __post_init__(self):
        if self.created_at is None:
            self.created_at = datetime.now()

@dataclass
class Price:
    product_id: str
    retailer: str
    price: float
    currency: str
    availability: bool
    url: str
    timestamp: datetime = None
    shipping_cost: float = 0.0
    discount_percentage: float = 0.0

    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now()

# Colab-Optimized Database Manager
class ColabDatabaseManager:
    def __init__(self, db_path: str = "/content/ecommerce_comparison.db"):
        self.db_path = db_path
        self.init_database()
        print(f"✅ Database initialized at: {db_path}")

    def init_database(self):
        """Initialize database with required tables"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Products table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                category TEXT,
                brand TEXT,
                description TEXT,
                image_url TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # Prices table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS prices (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_id TEXT,
                retailer TEXT,
                price REAL,
                currency TEXT,
                availability BOOLEAN,
                url TEXT,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                shipping_cost REAL DEFAULT 0.0,
                discount_percentage REAL DEFAULT 0.0,
                FOREIGN KEY (product_id) REFERENCES products (id)
            )
        ''')

        # Create indexes for better performance
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_prices_product_timestamp ON prices(product_id, timestamp)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_prices_retailer ON prices(retailer)')

        conn.commit()
        conn.close()

    def save_product(self, product: Product):
        """Save product to database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT OR REPLACE INTO products
            (id, name, category, brand, description, image_url)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (product.id, product.name, product.category, product.brand,
              product.description, product.image_url))
        conn.commit()
        conn.close()

    def save_price(self, price: Price):
        """Save price to database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO prices
            (product_id, retailer, price, currency, availability, url, shipping_cost, discount_percentage)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (price.product_id, price.retailer, price.price, price.currency,
              price.availability, price.url, price.shipping_cost, price.discount_percentage))
        conn.commit()
        conn.close()

    def get_price_history(self, product_id: str, days: int = 7) -> pd.DataFrame:
        """Get price history as DataFrame"""
        conn = sqlite3.connect(self.db_path)
        query = '''
            SELECT retailer, price, timestamp, availability, shipping_cost, discount_percentage
            FROM prices
            WHERE product_id = ? AND timestamp >= datetime('now', '-{} days')
            ORDER BY timestamp DESC
        '''.format(days)

        df = pd.read_sql_query(query, conn, params=(product_id,))
        conn.close()

        if not df.empty:
            df['timestamp'] = pd.to_datetime(df['timestamp'])

        return df

    def get_all_products(self) -> pd.DataFrame:
        """Get all products as DataFrame"""
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql_query('SELECT * FROM products', conn)
        conn.close()
        return df

    def get_current_prices(self, product_id: str) -> pd.DataFrame:
        """Get latest prices for a product"""
        conn = sqlite3.connect(self.db_path)
        query = '''
            SELECT p1.* FROM prices p1
            INNER JOIN (
                SELECT product_id, retailer, MAX(timestamp) as max_timestamp
                FROM prices
                WHERE product_id = ?
                GROUP BY product_id, retailer
            ) p2 ON p1.product_id = p2.product_id
            AND p1.retailer = p2.retailer
            AND p1.timestamp = p2.max_timestamp
            ORDER BY p1.price ASC
        '''

        df = pd.read_sql_query(query, conn, params=(product_id,))
        conn.close()

        if not df.empty:
            df['timestamp'] = pd.to_datetime(df['timestamp'])

        return df

# Simplified Web Scrapers for Colab
class ColabScraper:
    def __init__(self, name: str, base_url: str):
        self.name = name
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })

    def clean_price(self, price_text: str) -> float:
        """Extract numeric price from text"""
        if not price_text:
            return 0.0

        # Remove currency symbols and extract numbers
        price_match = re.search(r'[\d,]+\.?\d*', str(price_text).replace(',', ''))
        return float(price_match.group()) if price_match else 0.0

    def search_products(self, query: str, max_results: int = 5) -> List[Dict]:
        """Search for products (mock implementation for demo)"""
        # In a real implementation, this would scrape actual websites
        # For Colab demo, we'll generate sample data

        products = []
        retailers = ['RetailMart', 'ShopCenter', 'BargainHub', 'SuperStore', 'MegaShop']
        categories = ['Electronics', 'Clothing', 'Books', 'Home & Garden', 'Sports']

        for i in range(max_results):
            product_id = hashlib.md5(f"{query}_{self.name}_{i}".encode()).hexdigest()[:8]
            base_price = np.random.uniform(10, 500)

            product = {
                'id': product_id,
                'name': f"{query} - {self.name} Model {i+1}",
                'category': np.random.choice(categories),
                'brand': f"Brand{i+1}",
                'description': f"High-quality {query} with excellent features",
                'image_url': f"https://via.placeholder.com/200x200?text={query}",
                'price': round(base_price * np.random.uniform(0.8, 1.2), 2),
                'retailer': self.name,
                'currency': 'USD',
                'availability': np.random.choice([True, False], p=[0.8, 0.2]),
                'url': f"https://{self.name.lower()}.com/product/{product_id}",
                'shipping_cost': round(np.random.uniform(0, 15), 2),
                'discount_percentage': round(np.random.uniform(0, 0.3), 2)
            }
            products.append(product)

        return products

# AI Recommendation Engine for Colab
class ColabRecommendationEngine:
    def __init__(self, db_manager: ColabDatabaseManager):
        self.db = db_manager
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
        self.product_features = None
        self.product_ids = []

    def build_product_features(self):
        """Build feature vectors for products"""
        df = self.db.get_all_products()

        if df.empty:
            print("⚠️ No products in database yet. Add some products first.")
            return

        # Combine text features
        texts = []
        self.product_ids = []

        for _, row in df.iterrows():
            text = f"{row['name']} {row['description']} {row['category']} {row['brand']}"
            texts.append(text)
            self.product_ids.append(row['id'])

        # Build TF-IDF vectors
        try:
            self.product_features = self.vectorizer.fit_transform(texts)
            print(f"✅ Built features for {len(texts)} products")
        except Exception as e:
            print(f"❌ Error building features: {e}")

    def get_similar_products(self, product_id: str, n_recommendations: int = 3) -> List[Dict]:
        """Get similar products"""
        if self.product_features is None or product_id not in self.product_ids:
            return []

        # Find product index
        product_idx = self.product_ids.index(product_id)

        # Calculate similarities
        similarities = cosine_similarity(
            self.product_features[product_idx:product_idx+1],
            self.product_features
        ).flatten()

        # Get top recommendations
        similar_indices = similarities.argsort()[-n_recommendations-1:-1][::-1]

        recommendations = []
        df = self.db.get_all_products()

        for idx in similar_indices:
            if idx != product_idx:
                product_data = df[df['id'] == self.product_ids[idx]].iloc[0]
                recommendations.append({
                    'id': product_data['id'],
                    'name': product_data['name'],
                    'category': product_data['category'],
                    'similarity_score': float(similarities[idx])
                })

        return recommendations

# Analytics Engine for Colab
class ColabAnalyticsEngine:
    def __init__(self, db_manager: ColabDatabaseManager):
        self.db = db_manager

    def generate_price_comparison_chart(self, product_id: str):
        """Generate price comparison visualization"""
        df = self.db.get_current_prices(product_id)

        if df.empty:
            print("❌ No price data available for this product")
            return

        # Create comparison chart
        fig = px.bar(
            df,
            x='retailer',
            y='price',
            color='availability',
            title=f'Price Comparison Across Retailers',
            labels={'price': 'Price (USD)', 'retailer': 'Retailer'},
            color_discrete_map={True: 'green', False: 'red'}
        )

        fig.update_layout(height=400, showlegend=True)
        fig.show()

        # Display summary statistics
        print("\n📊 Price Summary:")
        print(f"Lowest Price: ${df['price'].min():.2f}")
        print(f"Highest Price: ${df['price'].max():.2f}")
        print(f"Average Price: ${df['price'].mean():.2f}")
        print(f"Available at {df['availability'].sum()} out of {len(df)} retailers")

    def generate_price_history_chart(self, product_id: str, days: int = 7):
        """Generate price history visualization"""
        df = self.db.get_price_history(product_id, days)

        if df.empty:
            print("❌ No price history available for this product")
            return

        # Create line chart
        fig = px.line(
            df,
            x='timestamp',
            y='price',
            color='retailer',
            title=f'Price History - Last {days} Days',
            labels={'price': 'Price (USD)', 'timestamp': 'Date'}
        )

        fig.update_layout(height=400)
        fig.show()

    def generate_market_overview(self):
        """Generate market overview dashboard"""
        conn = sqlite3.connect(self.db.db_path)

        # Get market statistics
        market_stats = pd.read_sql_query('''
            SELECT
                p.category,
                COUNT(DISTINCT p.id) as product_count,
                AVG(pr.price) as avg_price,
                MIN(pr.price) as min_price,
                MAX(pr.price) as max_price
            FROM products p
            LEFT JOIN prices pr ON p.id = pr.product_id
            WHERE pr.price IS NOT NULL
            GROUP BY p.category
        ''', conn)

        retailer_stats = pd.read_sql_query('''
            SELECT
                retailer,
                COUNT(*) as product_count,
                AVG(price) as avg_price,
                SUM(availability) as available_products
            FROM prices
            GROUP BY retailer
        ''', conn)

        conn.close()

        if not market_stats.empty:
            # Category overview
            fig1 = px.bar(
                market_stats,
                x='category',
                y='avg_price',
                title='Average Price by Category',
                labels={'avg_price': 'Average Price (USD)'}
            )
            fig1.show()

            # Product count by category
            fig2 = px.pie(
                market_stats,
                values='product_count',
                names='category',
                title='Product Distribution by Category'
            )
            fig2.show()

        if not retailer_stats.empty:
            # Retailer comparison
            fig3 = px.scatter(
                retailer_stats,
                x='product_count',
                y='avg_price',
                size='available_products',
                hover_name='retailer',
                title='Retailer Analysis: Product Count vs Average Price',
                labels={'product_count': 'Number of Products', 'avg_price': 'Average Price (USD)'}
            )
            fig3.show()

# Main Colab Price Comparison System
class ColabPriceComparisonSystem:
    def __init__(self):
        self.db = ColabDatabaseManager()
        self.scrapers = {
            'RetailMart': ColabScraper('RetailMart', 'https://retailmart.com'),
            'ShopCenter': ColabScraper('ShopCenter', 'https://shopcenter.com'),
            'BargainHub': ColabScraper('BargainHub', 'https://bargainhub.com')
        }
        self.recommendation_engine = ColabRecommendationEngine(self.db)
        self.analytics = ColabAnalyticsEngine(self.db)

        print("✅ Price Comparison System Initialized!")
        print(f"📊 Available retailers: {list(self.scrapers.keys())}")

    def search_and_compare(self, query: str, max_results_per_retailer: int = 3):
        """Search products across all retailers and save to database"""
        print(f"🔍 Searching for '{query}' across all retailers...")

        all_products = []

        # Search across all scrapers
        for retailer_name, scraper in self.scrapers.items():
            print(f"  📍 Searching {retailer_name}...")
            try:
                products = scraper.search_products(query, max_results_per_retailer)
                all_products.extend(products)
                print(f"    ✅ Found {len(products)} products")
            except Exception as e:
                print(f"    ❌ Error searching {retailer_name}: {e}")

        # Save products and prices to database
        saved_products = {}
        for product_data in all_products:
            # Create product
            product = Product(
                id=product_data['id'],
                name=product_data['name'],
                category=product_data['category'],
                brand=product_data['brand'],
                description=product_data['description'],
                image_url=product_data['image_url']
            )

            # Save product
            self.db.save_product(product)

            # Create and save price
            price = Price(
                product_id=product_data['id'],
                retailer=product_data['retailer'],
                price=product_data['price'],
                currency=product_data['currency'],
                availability=product_data['availability'],
                url=product_data['url'],
                shipping_cost=product_data['shipping_cost'],
                discount_percentage=product_data['discount_percentage']
            )

            self.db.save_price(price)

            # Group by product name for comparison
            if product.name not in saved_products:
                saved_products[product.name] = []
            saved_products[product.name].append(product_data)

        print(f"\n✅ Saved {len(all_products)} products to database")

        # Display results
        self.display_search_results(saved_products)

        # Rebuild recommendation features
        self.recommendation_engine.build_product_features()

        return saved_products

    def display_search_results(self, products_dict: Dict):
        """Display search results in a formatted way"""
        print("\n" + "="*80)
        print("🛒 SEARCH RESULTS")
        print("="*80)

        for product_name, product_list in products_dict.items():
            print(f"\n📦 {product_name}")
            print("-" * 60)

            # Sort by price
            sorted_products = sorted(product_list, key=lambda x: x['price'])

            for i, product in enumerate(sorted_products):
                status = "✅ Available" if product['availability'] else "❌ Out of Stock"
                discount = f" ({product['discount_percentage']*100:.0f}% off)" if product['discount_percentage'] > 0 else ""

                print(f"  {i+1}. {product['retailer']}")
                print(f"     💰 ${product['price']:.2f}{discount}")
                print(f"     🚚 Shipping: ${product['shipping_cost']:.2f}")
                print(f"     📊 {status}")
                print(f"     🔗 {product['url']}")
                print()

    def get_product_recommendations(self, product_id: str):
        """Get recommendations for a product"""
        recommendations = self.recommendation_engine.get_similar_products(product_id)

        if recommendations:
            print(f"\n🎯 SIMILAR PRODUCTS:")
            print("-" * 40)
            for i, rec in enumerate(recommendations, 1):
                print(f"{i}. {rec['name']}")
                print(f"   Category: {rec['category']}")
                print(f"   Similarity: {rec['similarity_score']:.2f}")
                print()
        else:
            print("❌ No recommendations available")

    def generate_analytics_dashboard(self):
        """Generate comprehensive analytics dashboard"""
        print("\n📊 ANALYTICS DASHBOARD")
        print("="*50)

        # Market overview
        self.analytics.generate_market_overview()

        # Get a sample product for demonstration
        df = self.db.get_all_products()
        if not df.empty:
            sample_product_id = df.iloc[0]['id']
            print(f"\n📈 Sample Analysis for Product: {df.iloc[0]['name']}")
            self.analytics.generate_price_comparison_chart(sample_product_id)

            # Add some historical data for demo
            self.simulate_price_history(sample_product_id)
            self.analytics.generate_price_history_chart(sample_product_id)

    def simulate_price_history(self, product_id: str, days: int = 7):
        """Simulate price history for demonstration"""
        current_prices = self.db.get_current_prices(product_id)

        if current_prices.empty:
            return

        # Generate historical prices
        for day in range(1, days + 1):
            timestamp = datetime.now() - timedelta(days=day)

            for _, current_price in current_prices.iterrows():
                # Simulate price fluctuation
                price_variation = np.random.uniform(0.9, 1.1)
                historical_price = current_price['price'] * price_variation

                # Create historical price entry
                historical_price_obj = Price(
                    product_id=product_id,
                    retailer=current_price['retailer'],
                    price=round(historical_price, 2),
                    currency=current_price['currency'],
                    availability=np.random.choice([True, False], p=[0.8, 0.2]),
                    url=current_price['url'],
                    timestamp=timestamp,
                    shipping_cost=current_price['shipping_cost'],
                    discount_percentage=current_price['discount_percentage']
                )

                self.db.save_price(historical_price_obj)

# Interactive Colab Interface
def create_colab_interface():
    """Create interactive widgets for Colab"""
    system = ColabPriceComparisonSystem()

    # Search interface
    search_widget = widgets.Text(
        value='laptop',
        placeholder='Enter product to search...',
        description='Search:',
        style={'description_width': 'initial'}
    )

    max_results_widget = widgets.IntSlider(
        value=3,
        min=1,
        max=10,
        step=1,
        description='Results per retailer:',
        style={'description_width': 'initial'}
    )

    search_button = widgets.Button(
        description='🔍 Search Products',
        button_style='primary',
        tooltip='Search for products across all retailers'
    )

    analytics_button = widgets.Button(
        description='📊 Generate Analytics',
        button_style='success',
        tooltip='Generate analytics dashboard'
    )

    output = widgets.Output()

    def on_search_clicked(b):
        with output:
            clear_output()
            query = search_widget.value
            max_results = max_results_widget.value

            if query:
                system.search_and_compare(query, max_results)
            else:
                print("❌ Please enter a search query")

    def on_analytics_clicked(b):
        with output:
            clear_output()
            system.generate_analytics_dashboard()

    search_button.on_click(on_search_clicked)
    analytics_button.on_click(on_analytics_clicked)

    # Layout
    search_box = widgets.VBox([
        widgets.HTML("<h2>🛒 E-commerce Price Comparison Platform</h2>"),
        search_widget,
        max_results_widget,
        widgets.HBox([search_button, analytics_button]),
        output
    ])

    return search_box, system

# Demo Data Generator
def generate_demo_data(system: ColabPriceComparisonSystem):
    """Generate demo data for testing"""
    print("🎭 Generating demo data...")

    demo_queries = ['laptop', 'smartphone', 'headphones', 'tablet', 'camera']

    for query in demo_queries:
        print(f"  📱 Adding {query} products...")
        system.search_and_compare(query, 2)
        time.sleep(0.5)  # Small delay to simulate real scraping

    print("✅ Demo data generated successfully!")

# Main execution for Colab
def run_colab_price_comparison():
    """Main function to run the price comparison system in Colab"""

    print("""
    🎯 E-commerce Price Comparison Platform - Colab Edition
    ======================================================

    ✨ Features:
    • Multi-retailer price comparison
    • Real-time data visualization
    • AI-powered recommendations
    • Historical price tracking
    • Interactive analytics dashboard
    • Market trend analysis

    🚀 Ready to start comparing prices!
    """)

    # Create interactive interface
    interface, system = create_colab_interface()

    # Generate some demo data
    generate_demo_data(system)

    # Display interface
    display(interface)

    return system

# Additional utility functions for Colab
def export_data_to_csv(system: ColabPriceComparisonSystem, filename: str = "price_comparison_data.csv"):
    """Export all data to CSV for download"""
    conn = sqlite3.connect(system.db.db_path)

    query = '''
        SELECT
            p.name as product_name,
            p.category,
            p.brand,
            pr.retailer,
            pr.price,
            pr.currency,
            pr.availability,
            pr.shipping_cost,
            pr.discount_percentage,
            pr.timestamp
        FROM products p
        JOIN prices pr ON p.id = pr.product_id
        ORDER BY p.name, pr.price
    '''

    df = pd.read_sql_query(query, conn)
    conn.close()

    df.to_csv(f"/content/{filename}", index=False)
    print(f"✅ Data exported to /content/{filename}")

    return df

def create_advanced_analytics(system: ColabPriceComparisonSystem):
    """Create advanced analytics visualizations"""
    conn = sqlite3.connect(system.db.db_path)

    # Price distribution analysis
    price_data = pd.read_sql_query('''
        SELECT p.category, pr.price, pr.retailer
        FROM products p
        JOIN prices pr ON p.id = pr.product_id
        WHERE pr.availability = 1
    ''', conn)

    if not price_data.empty:
        # Price distribution by category
        fig1 = px.box(
            price_data,
            x='category',
            y='price',
            title='Price Distribution by Category',
            labels={'price': 'Price (USD)'}
        )
        fig1.show()

        # Average price by retailer
        retailer_avg = price_data.groupby('retailer')['price'].mean().reset_index()
        fig2 = px.bar(
            retailer_avg,
            x='retailer',
            y='price',
            title='Average Price by Retailer',
            labels={'price': 'Average Price (USD)'}
        )
        fig2.show()

        # Price correlation heatmap
        pivot_data = price_data.pivot_table(
            values='price',
            index='category',
            columns='retailer',
            aggfunc='mean'
        )

        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='RdYlBu_r')
        plt.title('Average Price Heatmap: Category vs Retailer')
        plt.tight_layout()
        plt.show()

    conn.close()

# Run the system
print("🎬 Starting E-commerce Price Comparison Platform...")
system = run_colab_price_comparison()

print("""
🎉 Platform is ready!

📝 Usage Instructions:
1. Use the search box above to find products
2. Click "Search Products" to compare prices
3. Click "Generate Analytics" for detailed insights
4. All data is automatically saved to SQLite database

💡 Additional Features:
• export_data_to_csv(system) - Export data to CSV
• create_advanced_analytics(system) - Advanced visualizations
• system.analytics.generate_market_overview() - Market analysis

🔗 Database location: /content/ecommerce_comparison.db
""")

🚀 E-commerce Price Comparison Platform for Google Colab
🎬 Starting E-commerce Price Comparison Platform...

    🎯 E-commerce Price Comparison Platform - Colab Edition
    
    ✨ Features:
    • Multi-retailer price comparison
    • Real-time data visualization
    • AI-powered recommendations
    • Historical price tracking
    • Interactive analytics dashboard
    • Market trend analysis
    
    🚀 Ready to start comparing prices!
    
✅ Database initialized at: /content/ecommerce_comparison.db
✅ Price Comparison System Initialized!
📊 Available retailers: ['RetailMart', 'ShopCenter', 'BargainHub']
🎭 Generating demo data...
  📱 Adding laptop products...
🔍 Searching for 'laptop' across all retailers...
  📍 Searching RetailMart...
    ✅ Found 2 products
  📍 Searching ShopCenter...
    ✅ Found 2 products
  📍 Searching BargainHub...
    ✅ Found 2 products

✅ Saved 6 products to database

🛒 SEARCH RESULTS

📦 laptop - RetailMart Model 1
-------------------------------------------------------


🎉 Platform is ready!

📝 Usage Instructions:
1. Use the search box above to find products
2. Click "Search Products" to compare prices
3. Click "Generate Analytics" for detailed insights
4. All data is automatically saved to SQLite database

💡 Additional Features:
• export_data_to_csv(system) - Export data to CSV
• create_advanced_analytics(system) - Advanced visualizations
• system.analytics.generate_market_overview() - Market analysis

🔗 Database location: /content/ecommerce_comparison.db

